In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import time, os
from sklearn.metrics import mean_squared_error

## DANE

In [2]:
# ładowanie danych treningowych
X = pd.read_csv('X_train.csv', header=0)
y = pd.read_csv('y_train.csv', header=0)['Expected']

In [3]:
# podział danych (X_test i y_test posłużą jako dane walidacyjne)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# BASELINE model

In [11]:
%%time
base = GradientBoostingRegressor()
base.fit(X_train,y_train)
mean_squared_error(y_test, base.predict(X_test))
# 0.31999768147616214

CPU times: total: 4min 24s
Wall time: 4min 26s


0.34620508882151935

## Scaling data

In [5]:
%%time
sc_X = StandardScaler()
sc_X.fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)
X = sc_X.transform(X)

CPU times: total: 1.45 s
Wall time: 1.48 s


In [6]:
# %%time
# pca = PCA()
# pca.fit(X)
# pca_data = pca.transform(X)
# pca.explained_variance_ratio_ * 100

## GradientBoosting

In [7]:
%%time
#siatka parametrów do modelu
parameters = {
    'pca__n_components': [50, 100, 200],
    'gradientboosting__learning_rate': [0.05, 0.1],
    'gradientboosting__n_estimators': [100, 200, 500],
    'gradientboosting__subsample': [0.8, 1.0],
    'gradientboosting__max_depth': [3, 5],
    'gradientboosting__min_samples_split': [2, 4],
    'gradientboosting__min_samples_leaf': [1, 2],
    'gradientboosting__max_features': ['auto'],
}
# model
pipeline = Pipeline([
    ('pca', PCA()),
    ('gradientboosting', GradientBoostingRegressor())
])

# cross walidacja
start = time.time()
reg = GridSearchCV(estimator=pipeline, param_grid=parameters, scoring='neg_root_mean_squared_error',
                   n_jobs=os.cpu_count(), verbose=4, cv=3)
reg.fit(X_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


C:\Users\ChecDoNauki\PycharmProjects\SUS2023\venv\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C:\Users\ChecDoNauki\PycharmProjects\SUS2023\venv\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C:\Users\ChecDoNauki\PycharmProjects\SUS2023\venv\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C:\Users\ChecDoNauki\PycharmProjects\SUS2023\venv\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.

CPU times: total: 28.8 s
Wall time: 2h 59min 28s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('gradientboosting',
                                        GradientBoostingRegressor())]),
             n_jobs=8,
             param_grid={'gradientboosting__learning_rate': [0.05, 0.1],
                         'gradientboosting__max_depth': [3, 5],
                         'gradientboosting__max_features': ['auto'],
                         'gradientboosting__min_samples_leaf': [1, 2],
                         'gradientboosting__min_samples_split': [2, 4],
                         'gradientboosting__n_estimators': [100, 200, 500],
                         'gradientboosting__subsample': [0.8, 1.0],
                         'pca__n_components': [50, 100, 200]},
             scoring='neg_root_mean_squared_error', verbose=4)

In [9]:
reg.best_params_
# {'gradientboosting__learning_rate': 0.05,
#  'gradientboosting__max_depth': 5,
#  'gradientboosting__max_features': 'auto',
#  'gradientboosting__min_samples_leaf': 1,
#  'gradientboosting__min_samples_split': 2,
#  'gradientboosting__n_estimators': 100,
#  'gradientboosting__subsample': 0.8,
#  'pca__n_components': 50}

{'gradientboosting__learning_rate': 0.05,
 'gradientboosting__max_depth': 5,
 'gradientboosting__max_features': 'auto',
 'gradientboosting__min_samples_leaf': 1,
 'gradientboosting__min_samples_split': 2,
 'gradientboosting__n_estimators': 100,
 'gradientboosting__subsample': 0.8,
 'pca__n_components': 50}

In [10]:
mean_squared_error(y_test, reg.predict(X_test))

0.36932317950166127